In [ ]:
# Imports and installs
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from zipfile import ZipFile # library for handling ZIP files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge geocoder --yes
import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from folium import plugins

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Download Dutch area geolocation data based on postal code

Dutch postal codes have 6 positions: four digits followed by two letters. The four digits are the best Dutch proxy of neighbourhoods.

Geolocation data sets for the four digits postal code (4PP) can amongst others be found at https://git.tuxm.nl/tuxmachine/postcodes/src/4329c858db24b79523fd3fbbaf2df138ccaf16cd

Credit: https://git.tuxm.nl/tuxmachine/postcodes/src/master/README.md

License: https://git.tuxm.nl/tuxmachine/postcodes/src/master/LICENSE

In [ ]:
#4PP Polygons in JSON dataset for Netherlands areas in polygon geolocation
!wget -q -O '4pp.polygons.json' https://git.tuxm.nl/tuxmachine/postcodes/raw/4329c858db24b79523fd3fbbaf2df138ccaf16cd/4pp.polygons.json
print('Data downloaded!')

In [ ]:
#4PP LatLong in CSV dataset for Netherlands areas in LatLong geolocation
!wget -q -O '4pp.csv' https://git.tuxm.nl/tuxmachine/postcodes/src/master/4pp.csv
print('Data downloaded!')

In [ ]:
neighlatlong = pd.read_csv('https://git.tuxm.nl/tuxmachine/postcodes/raw/master/4pp.csv')
neighlatlong.head()

In [ ]:
neighlatlong.shape

Drop all areas with category ('Soort') Postbus (=P.O. Box) or Onbekend (=Unkown)

In [ ]:
#Drop all rows with Soort is Onbekend
neighlatlong.drop(neighlatlong[neighlatlong.soort == 'Onbekend'].index, inplace=True)
#Drop all rows with Soort is Postbus
neighlatlong.drop(neighlatlong[neighlatlong.soort == 'Postbus'].index, inplace=True)
neighlatlong.head()

In [ ]:
neighlatlong.shape

### CBS names of areas and neighbourhoods based on postal code

The Dutch Central Bureau of Statistics (CBS) has an open dataset on areas and neighbourhoods based on postal code.

See: https://www.cbs.nl/nl-nl/maatwerk/2018/36/buurt-wijk-en-gemeente-2018-voor-postcode-huisnummer

In [ ]:
#Download and extract ZIP file of CBS area and neighbourhood data
!wget -q -O '2018-cbs-pc6huisnr20180801_buurt-vs2.zip' https://www.cbs.nl/-/media/_excel/2018/36/2018-cbs-pc6huisnr20180801_buurt%20-vs2.zip
with ZipFile('2018-cbs-pc6huisnr20180801_buurt-vs2.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()
print('Data downloaded!')

In [ ]:
#Create dataframe on lowlevel postal code data
pc = pd.read_csv('pc6hnr20180801_gwb-vs2.csv', sep=';', encoding='latin_1')
pc.rename(columns = {'Buurt2018': 'buurtcode', 'Wijk2018': 'wijkcode', 'Gemeente2018': 'gemeentecode'}, inplace = True)
pc.head()

In [ ]:
#Create dataframe with neighbourhood codes and names
wijk = pd.read_csv('wijknaam2018.csv', sep=';', encoding='latin_1')
wijk.rename(columns = {'GWBcode8': 'wijkcode', 'GWBlabel': 'wijk'}, inplace = True)
wijk.head()

In [ ]:
#Create dataframe with area codes and names
buurt = pd.read_csv('buurtnaam2018.csv', sep=';', encoding='latin_1')
buurt.rename(columns = {'GWBcode8': 'buurtcode', 'GWBlabel': 'buurt'}, inplace = True)
buurt.head()

In [ ]:
#Merge postal code dataframe with neighbourhood and area names
pcmerge = pd.merge(pc, wijk, on='wijkcode')
pcmerge = pd.merge(pcmerge, buurt, on='buurtcode')
pcmerge.head()

In [ ]:
#Drop columns we will not need: housenumber (too low level), Municipality (will be added by geocode data by name instead of code) and the codes
pcmerge.drop(['Huisnummer', 'buurtcode', 'wijkcode', 'gemeentecode'], axis=1, inplace = True)
pcmerge.head()

In [ ]:
#Create column with first four digits of postal code as all six is too low level
pcmerge['postcode'] = pcmerge['PC6'].str[:4]
pcmerge.head()

In [ ]:
#Drop PC6 and group on postcode
postcode = pcmerge.groupby(['postcode'], as_index=False).first()
postcode.drop(['PC6'], axis=1, inplace = True)
postcode.head()

In [ ]:
#Cast column postcode to INT instead of OBJECT as geolocation dataset has INT
postcode.postcode = postcode.postcode.astype('int64')

In [ ]:
#Now merge with long/lat
postcode = pd.merge(postcode, neighlatlong, on='postcode')
postcode.head()

In [ ]:
#Drop unneeded columns
postcode.drop(['id', 'alternatieve_schrijfwijzen', 'soort'], axis=1, inplace = True)
postcode.head()

In [ ]:
postcode.shape

### Show neighbourhoods on map for the city of Utrecht

In [ ]:
# Obtain latitude and longitude of the Netherlands
g = geocoder.arcgis('Utrecht, Netherlands')
Utrecht_coords = g.latlng
Utrecht_coords

In [ ]:
#Extract only woonplaats is Utrecht
utrecht = postcode[postcode.woonplaats.str.contains('utrecht',case=False)]
utrecht.head()

In [ ]:
#Create map of Utrecht with neighbourhoods marked
map_utrecht = folium.Map(location=[Utrecht_coords[0], Utrecht_coords[1]], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(utrecht['latitude'], utrecht['longitude'], utrecht['wijk'], utrecht['buurt']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_utrecht)  
    
map_utrecht